### 전체 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

In [4]:
# 청약매물 목록 정보
df_estate_list = pd.read_csv('../datasets/한국부동산원_청약홈_APT 분양정보_20250225.csv', encoding='cp949')

# 청약 신청자, 당첨자 정보
df_applicant_list = pd.read_csv('../datasets/한국부동산원_지역별 청약 신청자 정보_20250131.csv', encoding='cp949')
df_winner_list = pd.read_csv('../datasets/한국부동산원_지역별 청약 당첨자 정보_20250131.csv', encoding='cp949')

# 청약 당첨가점, 경쟁률 정보
df_estate_winner_score_list = pd.read_csv('../datasets/한국부동산원_청약홈_APT 가점정보_20250225.csv', encoding='cp949')
df_estate_competitition_rate_list = pd.read_csv('../datasets/한국부동산원_청약홈_APT 경쟁률_20250225.csv', encoding='cp949')

### 병합 1) 청약 매물 목록 정보 + 신청자, 당첨자 정보

In [5]:
# 청약매물 목록 정보에서 청약접수시작월, 당첨자발표월 컬럼 추가

df_estate_list['청약접수시작월'] = pd.to_datetime(df_estate_list['청약접수시작일'], format='%Y-%m-%d').dt.strftime('%Y-%m')
df_estate_list['당첨자발표월'] = pd.to_datetime(df_estate_list['당첨자발표일'], format='%Y-%m-%d').dt.strftime('%Y-%m')
df_estate_list[['공급지역명', '청약접수시작월', '당첨자발표월']].head()

,공급지역명,청약접수시작월,당첨자발표월
0,울산,2025-03,2025-03
1,대구,2025-03,2025-03
2,광주,2025-03,2025-03
3,인천,2025-03,2025-03
4,충남,2025-03,2025-03


In [6]:
# 청약 신청자 정보 확인

df_applicant_list.head()

,연월,시도,30대 이하,40대,50대,60대 이상
0,2020-02,서울,32635,12302,5215,2953
1,2020-02,부산,9190,3649,2223,2010
2,2020-02,대구,30455,14673,7477,5815
3,2020-02,인천,7391,4739,2464,1339
4,2020-02,울산,13,4,1,2


In [7]:
# 청약 신청자 정보 칼럼 가공

df_applicant_list['연월'] = pd.to_datetime(df_applicant_list['연월']).dt.strftime('%Y-%m')

# 신청자 접두사 추가
df_applicant_list = df_applicant_list.rename(columns={
    '30대 이하': '신청자 30대 이하',
    '40대': '신청자 40대',
    '50대': '신청자 50대',
    '60대 이상': '신청자 60대 이상'
})

df_applicant_list.head()

,연월,시도,신청자 30대 이하,신청자 40대,신청자 50대,신청자 60대 이상
0,2020-02,서울,32635,12302,5215,2953
1,2020-02,부산,9190,3649,2223,2010
2,2020-02,대구,30455,14673,7477,5815
3,2020-02,인천,7391,4739,2464,1339
4,2020-02,울산,13,4,1,2


In [8]:
# 청약 목록 데이터프레임에 연월과 시도에 따라 신청자 데이터 병합

df_estate_list = pd.merge(
    df_estate_list, 
    df_applicant_list,
    left_on=['공급지역명', '청약접수시작월'], 
    right_on=['시도', '연월'],
    how='inner',
)

df_estate_list.drop(columns=['청약접수시작월','연월', '시도'], inplace=True)

df_estate_list.head()

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,모집공고홈페이지주소,당첨자발표월,신청자 30대 이하,신청자 40대,신청자 50대,신청자 60대 이상
0,2025000004,2025000004,양주 용암 영무 예다음 더퍼스트,1,APT,1,민영,0,분양주택,410,...,N,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-02,147,23,10,3
1,2024000396,2024000396,부천 중앙하이츠 심곡 102동,1,APT,1,민영,0,분양주택,410,...,N,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-02,147,23,10,3
2,2024000725,2024000725,세종 5-1 양우내안애 아스펜,1,APT,1,민영,0,분양주택,338,...,N,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-02,2007,1461,842,455
3,2024000751,2024000751,더샵 라비온드,1,APT,1,민영,0,분양주택,560,...,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-02,11831,6962,4109,2396
4,2024000742,2024000742,한화포레나 부산덕천3차,1,APT,1,민영,0,분양주택,600,...,Y,N,N,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,2025-02,155,30,14,10


In [9]:
# 청약 당첨자 정보 확인

df_winner_list.head()

,연월,시도,30대 이하,40대,50대,60대 이상
0,2020-02,서울,392,291,167,112
1,2020-02,부산,455,150,67,50
2,2020-02,대구,319,209,88,40
3,2020-02,인천,680,229,92,50
4,2020-02,울산,13,4,1,2


In [10]:
# 청약 당첨자 정보 칼럼 가공

df_winner_list['연월'] = pd.to_datetime(df_winner_list['연월']).dt.strftime('%Y-%m')

#  당첨자 접두사 추가
df_winner_list = df_winner_list.rename(columns={
    '30대 이하': '당첨자 30대 이하',
    '40대': '당첨자 40대',
    '50대': '당첨자 50대',
    '60대 이상': '당첨자 60대 이상'
})

df_winner_list.head()

,연월,시도,당첨자 30대 이하,당첨자 40대,당첨자 50대,당첨자 60대 이상
0,2020-02,서울,392,291,167,112
1,2020-02,부산,455,150,67,50
2,2020-02,대구,319,209,88,40
3,2020-02,인천,680,229,92,50
4,2020-02,울산,13,4,1,2


In [11]:
# 청약 목록 데이터프레임에 연월과 시도에 따라 당첨자 데이터 병합

df_estate_list = pd.merge(
    df_estate_list, 
    df_winner_list,
    left_on=['공급지역명', '당첨자발표월'], 
    right_on=['시도', '연월'],
    how='inner',
)

df_estate_list.drop(columns=['당첨자발표월','연월', '시도'], inplace=True)

df_estate_list.head()

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,수도권내민영공공주택지구,모집공고홈페이지주소,신청자 30대 이하,신청자 40대,신청자 50대,신청자 60대 이상,당첨자 30대 이하,당첨자 40대,당첨자 50대,당첨자 60대 이상
0,2024820019,2024820019,성남금토 A-4블록 신혼희망타운(공공분양)(본청약),10,신혼희망타운,3,국민,0,분양주택,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,147,23,10,3,51,11,7,3
1,2024820018,2024820018,의왕청계2 A1블록 신혼희망타운(공공분양)(본청약),10,신혼희망타운,3,국민,0,분양주택,410,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,147,23,10,3,51,11,7,3
2,2024000704,2024000704,부산에코델타시티 대방 엘리움 리버뷰,1,APT,1,민영,0,분양주택,600,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,155,30,14,10,86,19,8,3
3,2024000733,2024000733,세종 번암리 리치빌 3차,1,APT,3,국민,0,분양주택,338,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,25,5,2,0,296,248,105,49
4,2024000732,2024000732,세종 번암리 리치빌 2차,1,APT,3,국민,0,분양주택,338,...,N,https://www.applyhome.co.kr/ai/aia/selectAPTLt...,25,5,2,0,296,248,105,49


### 병합 2) 청약 매물 목록 정보 + 당첨가점, 경쟁률 정보

In [12]:
# 당첨가점 데이터 확인

df_estate_winner_score_list

,주택관리번호,공고번호,모델번호,주택형,거주코드,거주지역,최저당첨가점,최고당첨가점,평균당첨가점
0,2025000004,2025000004,1,059.9973A,1,해당지역,NaN,NaN,NaN
1,2025000004,2025000004,1,059.9973A,2,기타지역,NaN,NaN,NaN
2,2025000004,2025000004,2,059.9957B,1,해당지역,NaN,NaN,NaN
3,2025000004,2025000004,2,059.9957B,2,기타지역,NaN,NaN,NaN
4,2025000004,2025000004,3,075.9281A,1,해당지역,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
23090,2020000001,2020000001,1,059.0000H,2,기타지역,NaN,NaN,NaN
23091,2020000001,2020000001,2,084.0000H,1,해당지역,NaN,NaN,NaN
23092,2020000001,2020000001,2,084.0000H,2,기타지역,NaN,NaN,NaN
23093,2020000001,2020000001,3,084.0000N,1,해당지역,NaN,NaN,NaN


In [13]:
# 경쟁률 데이터 확인

df_estate_competitition_rate_list

,주택관리번호,공고번호,모델번호,주택형,공급세대수,순위,거주코드,거주지역,접수건수,경쟁률
0,2025000015,2025000015,1,060.4624,135,1,1,해당지역,31,(△104)
1,2025000015,2025000015,1,060.4624,135,1,2,기타지역,5,(△99)
2,2025000015,2025000015,1,060.4624,135,2,1,해당지역,8,(△91)
3,2025000015,2025000015,1,060.4624,135,2,2,기타지역,3,(△88)
4,2025000015,2025000015,2,084.3616,41,1,1,해당지역,41,1.00
...,...,...,...,...,...,...,...,...,...,...
44223,2020000001,2020000001,2,084.0000H,122,2,2,기타지역,0,NaN
44224,2020000001,2020000001,3,084.0000N,12,1,1,해당지역,3175,264.58
44225,2020000001,2020000001,3,084.0000N,12,1,2,기타지역,0,NaN
44226,2020000001,2020000001,3,084.0000N,12,2,1,해당지역,0,NaN


In [14]:
# 경쟁률 데이터에 당첨가점 데이터 병합

def apply_score(row, score_column):
    rank = row['순위']

    # 2순위 인 경우 nan값으로 채운다
    if rank == 2:
        return np.nan

    # 조건에 맞는 데이터 필터링
    matched_rows = df_estate_winner_score_list[
        (df_estate_winner_score_list['공고번호'] == row['공고번호']) &
        (df_estate_winner_score_list['모델번호'] == row['모델번호']) &
        (df_estate_winner_score_list['주택형'] == row['주택형'])
    ]

    # 조건에 맞는 행이 없을 경우 nan값으로 채운다
    if matched_rows.empty:
        return np.nan 

    # 존재하면 첫번째 행의 값 리턴
    return matched_rows.iloc[0][score_column]

score_column_list = ['최저당첨가점', '평균당첨가점', '최고당첨가점']

for score_column in score_column_list:
    df_estate_competitition_rate_list[score_column] = df_estate_competitition_rate_list.apply(apply_score, args=(score_column,),  axis=1)

df_estate_competitition_rate_list

,주택관리번호,공고번호,모델번호,주택형,공급세대수,순위,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점
0,2025000015,2025000015,1,060.4624,135,1,1,해당지역,31,(△104),NaN,NaN,NaN
1,2025000015,2025000015,1,060.4624,135,1,2,기타지역,5,(△99),NaN,NaN,NaN
2,2025000015,2025000015,1,060.4624,135,2,1,해당지역,8,(△91),NaN,NaN,NaN
3,2025000015,2025000015,1,060.4624,135,2,2,기타지역,3,(△88),NaN,NaN,NaN
4,2025000015,2025000015,2,084.3616,41,1,1,해당지역,41,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44223,2020000001,2020000001,2,084.0000H,122,2,2,기타지역,0,NaN,NaN,NaN,NaN
44224,2020000001,2020000001,3,084.0000N,12,1,1,해당지역,3175,264.58,NaN,NaN,NaN
44225,2020000001,2020000001,3,084.0000N,12,1,2,기타지역,0,NaN,NaN,NaN,NaN
44226,2020000001,2020000001,3,084.0000N,12,2,1,해당지역,0,NaN,NaN,NaN,NaN


In [15]:
# 청약 매물 목록 데이터에 청약 매물 항목별 경쟁률, 당첨가점 병합

df_estate_list= pd.merge(
    df_estate_list, 
    df_estate_competitition_rate_list, 
    on=['주택관리번호', '공고번호'], 
    how='inner'
).reset_index(drop=True)

df_estate_list

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,주택형,공급세대수,순위,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점
0,2024000704,2024000704,부산에코델타시티 대방 엘리움 리버뷰,1,APT,1,민영,0,분양주택,600,...,119.2348A,59,1,1,해당지역,13,(△46),NaN,NaN,NaN
1,2024000704,2024000704,부산에코델타시티 대방 엘리움 리버뷰,1,APT,1,민영,0,분양주택,600,...,119.2348A,59,1,2,기타지역,3,(△43),NaN,NaN,NaN
2,2024000704,2024000704,부산에코델타시티 대방 엘리움 리버뷰,1,APT,1,민영,0,분양주택,600,...,119.2348A,59,2,1,해당지역,26,(△17),NaN,NaN,NaN
3,2024000704,2024000704,부산에코델타시티 대방 엘리움 리버뷰,1,APT,1,민영,0,분양주택,600,...,119.2348A,59,2,2,기타지역,5,(△12),NaN,NaN,NaN
4,2024000704,2024000704,부산에코델타시티 대방 엘리움 리버뷰,1,APT,1,민영,0,분양주택,600,...,119.1887B,130,1,1,해당지역,6,(△124),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38313,2020000007,2020000007,학성동 동남하이빌아파트,1,APT,1,민영,0,분양주택,680,...,084.8313C,15,2,2,기타지역,1,(△14),NaN,NaN,NaN
38314,2020000007,2020000007,학성동 동남하이빌아파트,1,APT,1,민영,0,분양주택,680,...,084.8655D,24,1,1,해당지역,1,(△23),NaN,NaN,NaN
38315,2020000007,2020000007,학성동 동남하이빌아파트,1,APT,1,민영,0,분양주택,680,...,084.8655D,24,1,2,기타지역,2,(△21),NaN,NaN,NaN
38316,2020000007,2020000007,학성동 동남하이빌아파트,1,APT,1,민영,0,분양주택,680,...,084.8655D,24,2,1,해당지역,2,(△19),NaN,NaN,NaN


### 3) 병합 데이터 저장

In [1]:
df_estate_list.to_csv('병합_청약매물_목록_정보.csv', index=False, encoding='cp949')

NameError: name 'df_estate_list' is not defined

In [30]:
df_estate_list_merged = pd.read_csv('병합_청약매물_목록_정보.csv', encoding='cp949')
df_estate_list_merged = df_estate_list_merged[df_estate_list_merged['공급지역명'].isin(['서울', '경기', '인천'])].reset_index(drop=True)
df_estate_list_merged

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,주택형,공급세대수,순위,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점
0,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,1,1,해당지역,13,(△11),0.0,0.00,0.0
1,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,1,2,기타지역,147,13.36,0.0,0.00,0.0
2,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,2,1,해당지역,0,NaN,NaN,NaN,NaN
3,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9493A,24,2,2,기타지역,0,NaN,NaN,NaN,NaN
4,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,075.9347B,13,1,1,해당지역,27,2.08,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19625,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.9100,43,2,2,기타지역,0,NaN,NaN,NaN,NaN
19626,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.8900,22,1,1,해당지역,16,(△6),23.0,31.36,51.0
19627,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.8900,22,1,2,기타지역,138,23.00,23.0,31.36,51.0
19628,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,074.8900,22,2,1,해당지역,0,NaN,NaN,NaN,NaN


### 4) 공급금액 크롤링 후 추가

In [31]:
import requests
from bs4 import BeautifulSoup
import time

In [32]:
def get_estate_price(estate_id):
    query = f'houseManageNo={estate_id}&pblancNo={estate_id}'
    URL = f'https://www.applyhome.co.kr/ai/aia/selectAPTLttotPblancDetail.do?{query}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(URL, headers=headers)
    response.content

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(response.content, 'lxml')

    # 테이블 찾기
    table = soup.findAll('table')[-2]

    # DataFrame 생성
    df = pd.read_html(str(table), converters={0:str, 1:str, 2:str})[0]

    return df

df_estate_price = get_estate_price(2020000009)
df_estate_price

C:\Users\user\AppData\Local\Temp\ipykernel_10484\822424156.py:16: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table = soup.findAll('table')[-2]
C:\Users\user\AppData\Local\Temp\ipykernel_10484\822424156.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), converters={0:str, 1:str, 2:str})[0]


,주택형,공급금액(최고가 기준),2순위 청약금
0,074.9100,45600,청약통장으로 청약(청약금 없음)
1,074.8900,45600,청약통장으로 청약(청약금 없음)


In [33]:
estate_ids = df_estate_list_merged['공고번호'].unique().tolist()
estate_ids

# 빈 문자열 존재하여 제거거
df_estate_list_merged['주택형'] = df_estate_list_merged['주택형'].str.strip()

for estate_id in estate_ids:
    try:
        # 청약 매물 가격 정보 가져오기
        df_estate_price = get_estate_price(estate_id)

        # 청약 매물 목록 데이터에 공급금액(최고가 기준) 컬럼 추가
        for index in range(len(df_estate_price)):
            estate_type = str(df_estate_price.loc[index, '주택형']).strip()
            estate_price = float(df_estate_price.loc[index, '공급금액(최고가 기준)']) * 10000

            mask = (df_estate_list_merged['공고번호'] == estate_id) & (df_estate_list_merged['주택형'] == estate_type)
            df_estate_list_merged.loc[mask, '공급금액(최고가 기준)'] = estate_price
    except:
        print(f'{estate_id} 데이터를 가져오는데 실패했습니다.')

df_estate_list_merged

C:\Users\user\AppData\Local\Temp\ipykernel_10484\822424156.py:16: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table = soup.findAll('table')[-2]
C:\Users\user\AppData\Local\Temp\ipykernel_10484\822424156.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), converters={0:str, 1:str, 2:str})[0]
C:\Users\user\AppData\Local\Temp\ipykernel_10484\822424156.py:16: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  table = soup.findAll('table')[-2]
C:\Users\user\AppData\Local\Temp\ipykernel_10484\822424156.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table),

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,공급세대수,순위,거주코드,거주지역,접수건수,경쟁률,최저당첨가점,평균당첨가점,최고당첨가점,공급금액(최고가 기준)
0,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,24,1,1,해당지역,13,(△11),0.0,0.00,0.0,664170000.0
1,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,24,1,2,기타지역,147,13.36,0.0,0.00,0.0,664170000.0
2,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,24,2,1,해당지역,0,NaN,NaN,NaN,NaN,664170000.0
3,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,24,2,2,기타지역,0,NaN,NaN,NaN,NaN,664170000.0
4,2024000706,2024000706,의왕 월암지구 대방 디에트르 레이크파크,1,APT,1,민영,0,분양주택,410,...,13,1,1,해당지역,27,2.08,NaN,NaN,NaN,591150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19625,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,43,2,2,기타지역,0,NaN,NaN,NaN,NaN,456000000.0
19626,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,22,1,1,해당지역,16,(△6),23.0,31.36,51.0,456000000.0
19627,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,22,1,2,기타지역,138,23.00,23.0,31.36,51.0,456000000.0
19628,2020000009,2020000009,의왕 오전 동아루미체,1,APT,1,민영,0,분양주택,410,...,22,2,1,해당지역,0,NaN,NaN,NaN,NaN,456000000.0


In [34]:
df_estate_list_merged['공급금액(최고가 기준)'].isna().sum()

np.int64(0)

In [36]:
df_estate_list_merged.columns

Index(['주택관리번호', '공고번호', '주택명', '주택구분코드', '주택구분코드명', '주택상세구분코드', '주택상세구분코드명',
       '분양구분코드', '분양구분코드명', '공급지역코드', '공급지역명', '공급위치우편번호', '공급위치', '공급규모',
       '모집공고일', '청약접수시작일', '청약접수종료일', '특별공급접수시작일', '특별공급접수종료일', '해당지역1순위접수시작일',
       '해당지역1순위접수종료일', '경기지역1순위접수시작일', '경기지역1순위접수종료일', '기타지역1순위접수시작일',
       '기타지역1순위접수종료일', '해당지역2순위접수시작일', '해당지역2순위접수종료일', '경기지역2순위접수시작일',
       '경기지역2순위접수종료일', '기타지역2순위접수시작일', '기타지역2순위접수종료일', '당첨자발표일', '계약시작일',
       '계약종료일', '홈페이지주소', '건설업체명_시공사', '문의처', '사업주체명_시행사', '입주예정월', '투기과열지구',
       '조정대상지역', '분양가상한제', '정비사업', '공공주택지구', '대규모택지개발지구', '수도권내민영공공주택지구',
       '모집공고홈페이지주소', '신청자 30대 이하', '신청자 40대', '신청자 50대', '신청자 60대 이상',
       '당첨자 30대 이하', '당첨자 40대', '당첨자 50대', '당첨자 60대 이상', '모델번호', '주택형',
       '공급세대수', '순위', '거주코드', '거주지역', '접수건수', '경쟁률', '최저당첨가점', '평균당첨가점',
       '최고당첨가점', '공급금액(최고가 기준)'],
      dtype='object')

In [35]:
df_estate_list_merged.to_csv('청약매물_공급금액 (서울, 경기, 인천).csv', index=False, encoding='cp949')